# Librerías

In [ ]:
from collections import defaultdict
import pandas as pd
import recommenders

# Importar datos preprocesados

In [ ]:
track_features = pd.read_csv('./data/processed_track_features.csv')
sessions = pd.read_csv('./data/processed_sessions.csv')
display(track_features.head(3))

# Nuestro modelo

# Crear Top N verdadero por usuario

In [ ]:
# top_n_verdadero_por_usuario es un diccionario de la forma
# {usuario1: [canción1, canción2, ...], usuario2: [canción3, canción4,...], ...}
# que contiene las canciones con las que ha interactuado un usuario. Consideraremos que si el
# usuario ha interactuado con la canción, entonces la canción es relevante para el usuario.
# top_n_verdadero_por_usuario = defaultdict(list)
# for i in range(len(sessions)):
#     fila = sessions.iloc[i, :]
#     top_n_verdadero_por_usuario[fila['session_id']].append(fila['track_id'])